In [ ]:
#!pip install gradio llama-index-llms-ollama llama-index-tools-mcp ollama mcp nest_asyncio llama-index

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

llm = Ollama(model="llama3.2", request_timeout=120.0)
Settings.llm = llm

In [ ]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tools = McpToolSpec(client=mcp_client) 

In [ ]:
tools = await mcp_tools.to_tool_list_async()

In [ ]:
SYSTEM_PROMPT = """\
You are an AI assistant for Tool Calling.
Before you help a user, you need to work with tools to interact with Employee Database.
"""

In [ ]:
from llama_index.tools.mcp import McpToolSpec
from llama_index.core.agent.workflow import FunctionAgent

async def get_agent(tools: McpToolSpec):
    tools = await tools.to_tool_list_async()
    agent = FunctionAgent(
        name="Employee Information Provider Agent",
        description="An agent that can work with Our Employee Information System to give information about employees.",
        tools=tools,
        llm=llm,
        system_prompt=SYSTEM_PROMPT,
    )
    return agent

In [ ]:
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.workflow import Context

async def get_employees_information(prompt: str):
    agent = await get_agent(mcp_tools)
    handler = agent.run(prompt)
    response = await handler
    return str(response)

In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=get_employees_information,
    inputs=gr.Textbox(lines=3, label="Ask Employee Information"),
    outputs=gr.Textbox(lines=10, label="Employee Information"),
    title="Agent UI",
    description="Talk to the agent"
)

iface.launch()
